__Trend analysis of temperature anomalies over time and forcing for SimMod.__

__The code is adapted from:__ https://github.com/hausfath/OldModels/blob/master/notebooks/obs_temp_forcing_analysis.ipynb

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import numpy_indexed as npi
import pandas as pd
import os
import statsmodels.formula.api as smf
import statsmodels.api as sm
import warnings
warnings.filterwarnings(action='once')

forcing_file = '../data/preprocessed/forcing_ensemble_base1961-1990.npy'
temps_file = '../data/preprocessed/Observations_TAnom.csv'
models_file = '../data/preprocessed/SimMod_timeseries.csv'
interim_path = "../data/TCR_interim/" # save interim data here

In [5]:
forcings = np.load(forcing_file, allow_pickle=True).item()
annual_temps = pd.read_csv(temps_file, skiprows=10)
single_models = pd.read_csv(models_file)

timeframes = ([1970, 2000], [1971, 2000], [1972, 2000], [1975, 2010], [1977, 2017], 
              [1981, 2017], [1988, 2017], [1990, 2017], [1993, 2017], [1995, 2017], 
              [2001, 2017], [2007, 2017])
model_names = ['SimMod2.6', 'SimMod4.5', 'SimMod6.0', 'SimMod8.5']

single_models = single_models[single_models['year']>1879]
#single_models = single_models[single_models['year']<2020]
single_models = single_models.reset_index(drop=True)
model_years = ([1970, 2017], [1970, 2017], [1970, 2017], [1970, 2017])

In [6]:
def coef_arma_cis(y_data, x_data):
    '''
    Calculate coefficients and CIs using OLS
    '''
    X = x_data
    X = sm.add_constant(X)
    smresults = sm.OLS(y_data, X).fit()
    ols_coef = smresults.params[1]
    ols_ci = ols_coef - smresults.conf_int(alpha=0.05, cols=None)[0][1]
    ci_lower = smresults.conf_int(alpha=0.05, cols=None)[0][1]
    ci_upper = smresults.conf_int(alpha=0.05, cols=None)[1][1]
    sd = (ci_upper - ols_coef) / 2.
    return {
            'coef' : ols_coef,
            'ci_lower' : ci_lower,
            'ci_upper' : ci_upper,
            'sd' : sd
    }

def simmod_forcing_time_trends(single_models, model_names, model_years):
    coef, coef_low, coef_high, timeframe, model, dtype = [], [], [], [], [], []
    for i in range(len(model_names)):
        print('Analyzing '+model_names[i]+' from '+str(model_years[i][0])+' to '+str(model_years[i][1]))
        years = single_models['year'].between(model_years[i][0], model_years[i][1])
        temp_year = coef_arma_cis(single_models[model_names[i]+'_t'][years], single_models['year'][years])
        temp_forcing = coef_arma_cis(single_models[model_names[i]+'_t'][years], single_models[model_names[i]+'_f'][years])
        coef.append(temp_year['coef'])
        coef_low.append(temp_year['ci_lower'])
        coef_high.append(temp_year['ci_upper'])
        timeframe.append(str(model_years[i][0])+' to '+str(model_years[i][1]))
        model.append(model_names[i])
        dtype.append('model_time')
        coef.append(temp_forcing['coef'])
        coef_low.append(temp_forcing['ci_lower'])
        coef_high.append(temp_forcing['ci_upper'])
        timeframe.append(str(model_years[i][0])+' to '+str(model_years[i][1]))
        model.append(model_names[i])
        dtype.append('model_forcing')
    
    df = pd.DataFrame({'coef' : coef,
                       'coef_low' : coef_low,
                       'coef_high' : coef_high,
                       'timeframe' : timeframe,
                       'model' : model,
                       'dtype': dtype})
    df.to_csv(interim_path+'simmod_trends.csv')

def simmod_obs_time_diffs(single_models, model_names, model_years, annual_temps):
    coef_mean, coef_sd, ci_mean, timeframe, model = [], [], [], [], []
    for i in range(len(model_names)):
        coef, ci_lower, ci_upper = [], [], []
        years = single_models['year'].between(model_years[i][0], model_years[i][1])
        for obs_temps in (['hadcrut4', 'gistemp', 'noaa', 'berkeley', 'cowtan_way']):
            print('Analyzing '+model_names[i]+' '+obs_temps+' diffs from '+str(model_years[i][0])
                  +' to '+str(model_years[i][1]))
            model_obs_diff = single_models[model_names[i]+'_t'][years] - annual_temps[obs_temps][years]
            results = coef_arma_cis(model_obs_diff, single_models['year'][years])
            coef.append(results['coef'])
            ci_lower.append(results['ci_lower'])
            ci_upper.append(results['ci_upper'])
        df = pd.DataFrame({'coef' : coef,
                           'ci_lower' : ci_lower,
                           'ci_upper' : ci_upper})
        df['ci_val'] = df['coef'] - df['ci_lower']
        coef_mean.append(df['coef'].mean())
        coef_sd.append(df['coef'].std())
        ci_mean.append(df['ci_val'].mean())
        timeframe.append(str(model_years[i]))
        model.append(str(model_names[i]))
    
    df = pd.DataFrame({'coef_mean' : coef_mean,
                       'coef_sd' : coef_sd,
                       'ci_mean' : ci_mean,
                       'model' : model,
                       'timeframe' : timeframe})
    uncertainty = ((df['coef_sd']*2)**2 + df['ci_mean']**2)**(0.5)
    df['coef_low'] = df['coef_mean'] - uncertainty
    df['coef_high'] = df['coef_mean'] + uncertainty
    df.to_csv(interim_path+'simmod_obs_time_diffs.csv')    

simmod_forcing_time_trends(single_models, model_names, model_years)
simmod_obs_time_diffs(single_models, model_names, model_years, annual_temps)

Analyzing SimMod2.6 from 1970 to 2017
Analyzing SimMod4.5 from 1970 to 2017
Analyzing SimMod6.0 from 1970 to 2017
Analyzing SimMod8.5 from 1970 to 2017
Analyzing SimMod2.6 hadcrut4 diffs from 1970 to 2017
Analyzing SimMod2.6 gistemp diffs from 1970 to 2017
Analyzing SimMod2.6 noaa diffs from 1970 to 2017
Analyzing SimMod2.6 berkeley diffs from 1970 to 2017
Analyzing SimMod2.6 cowtan_way diffs from 1970 to 2017
Analyzing SimMod4.5 hadcrut4 diffs from 1970 to 2017
Analyzing SimMod4.5 gistemp diffs from 1970 to 2017
Analyzing SimMod4.5 noaa diffs from 1970 to 2017
Analyzing SimMod4.5 berkeley diffs from 1970 to 2017
Analyzing SimMod4.5 cowtan_way diffs from 1970 to 2017
Analyzing SimMod6.0 hadcrut4 diffs from 1970 to 2017
Analyzing SimMod6.0 gistemp diffs from 1970 to 2017
Analyzing SimMod6.0 noaa diffs from 1970 to 2017
Analyzing SimMod6.0 berkeley diffs from 1970 to 2017
Analyzing SimMod6.0 cowtan_way diffs from 1970 to 2017
Analyzing SimMod8.5 hadcrut4 diffs from 1970 to 2017
Analyzing

In [12]:
def output_forcing_ensemble(forcings, annual_temps, start_year, anom_period, path="./"):
    rf_year_range = np.where((forcings['year'] >= start_year) & (forcings['year'] <= 2017))[0]
    temp_year_range = np.where((annual_temps['year'] >= start_year) & (annual_temps['year'] <= 2017))[0]
    rf_anthro = forcings['rf_anthro'][rf_year_range]
    rf_anthro_anoms = rf_anthro - rf_anthro[0:anom_period].mean(axis=0)
    rf_anthro_anoms = np.swapaxes(rf_anthro_anoms,0,1)

    df_names = ('hadcrut4_df', 'gistemp_df', 'noaa_df', 'berkeley_df', 'cowtan_way_df')
    dfs ={}

    for df_names,obs_temps in zip(df_names, ['hadcrut4', 'gistemp', 'noaa', 'berkeley', 'cowtan_way']):
        dfs[df_names] = pd.DataFrame()
        dfs[df_names]['year'] = annual_temps['year'][temp_year_range]
        temps = annual_temps[obs_temps][temp_year_range].values
        anoms = temps - temps[0:anom_period].mean(axis=0)
        dfs[df_names]['temp'] = anoms
        for rf_num in range(1000):
            dfs[df_names]['forcing_'+str(rf_num)] = rf_anthro_anoms[rf_num]
        
        dfs[df_names].to_csv(path+obs_temps + '_' + 'forcings'+str(start_year)+'.csv')

#output_forcing_ensemble(forcings, annual_temps, 1970, 20, path="../data/processed/FAR_spaghetti/")
#output_forcing_ensemble(forcings, annual_temps, 1988, 20, path="../data/processed/Hansen_spaghetti/")

In [7]:
def simmod_forcing_rate(single_models, model_names, model_years):
    coef, coef_low, coef_high, timeframe, model = [], [], [], [], []
    for i in range(len(model_names)):
        print('Analyzing '+model_names[i]+' from '+str(model_years[i][0])+' to '+str(model_years[i][1]))
        years = single_models['year'].between(model_years[i][0], model_years[i][1])
        forcing_rate = coef_arma_cis(single_models[model_names[i]+'_f'][years], single_models['year'][years])
        coef.append(forcing_rate['coef'])
        coef_low.append(forcing_rate['ci_lower'])
        coef_high.append(forcing_rate['ci_upper'])
        timeframe.append(str(model_years[i][0])+' to '+str(model_years[i][1]))
        model.append(model_names[i])
    
    df = pd.DataFrame({'coef' : coef,
                       'coef_low' : coef_low,
                       'coef_high' : coef_high,
                       'timeframe' : timeframe,
                       'model' : model})
    df.to_csv(interim_path+'simmod_forcing_rate.csv')

simmod_forcing_rate(single_models, model_names, model_years)

Analyzing SimMod2.6 from 1970 to 2017
Analyzing SimMod4.5 from 1970 to 2017
Analyzing SimMod6.0 from 1970 to 2017
Analyzing SimMod8.5 from 1970 to 2017


In [8]:
def model_skill(obs_trend, pred_trend):
    skill = 1 - ((obs_trend - pred_trend)**2 / obs_trend**2) ** (0.5)
    return skill

def simmod_time_skill_scores(single_models, model_names, model_years, annual_temps):
    skill_mean, skill_median, skill_5th, skill_95th, timeframe, model = [], [], [], [], [], []
    for i in range(len(model_names)):
        skill = []
        years = single_models['year'].between(model_years[i][0], model_years[i][1])
        model_trend = coef_arma_cis(single_models[model_names[i]+'_t'][years], single_models['year'][years])
        for obs_temps in (['hadcrut4', 'gistemp', 'noaa', 'berkeley', 'cowtan_way']):
            print('Analyzing '+model_names[i]+' '+obs_temps+' diffs from '+str(model_years[i][0])+' to '+str(model_years[i][1]))
            obs_trend = coef_arma_cis(annual_temps[obs_temps][years], annual_temps['year'][years])
            model_monte_carlo = np.random.normal(model_trend['coef'], model_trend['sd'], 100)
            obs_monte_carlo = np.random.normal(obs_trend['coef'], obs_trend['sd'], 100)
            for j in range(100):
                skill.append(model_skill(obs_monte_carlo[j], model_monte_carlo[j]))
        df = pd.DataFrame({'skill' : skill})
        skill_mean.append(df['skill'].mean())
        skill_median.append(df['skill'].median())
        skill_5th.append(df['skill'].quantile(0.05))
        skill_95th.append(df['skill'].quantile(0.95))    
        timeframe.append(str(model_years[i]))
        model.append(str(model_names[i]))
    
    df = pd.DataFrame({'skill_mean' : skill_mean,
                       'skill_median' : skill_median,
                       'skill_5th' : skill_5th,
                       'skill_95th' : skill_95th,
                       'model' : model,
                       'timeframe' : timeframe})
    df.to_csv(interim_path+'simmod_time_skill_scores.csv')
    
def simmod_tcr_skill_scores(single_models, model_names, model_years, annual_temps, forcings):
    skill_mean, skill_median, skill_5th, skill_95th, timeframe, model = [], [], [], [], [], []
    rf_anthro = np.swapaxes(forcings['rf_anthro'],0,1)   
    for i in range(len(model_names)):
        print('Analyzing '+model_names[i]+' from '+str(model_years[i][0])+' to '+str(model_years[i][1]))
        skill = []
        years = single_models['year'].between(model_years[i][0], model_years[i][1])
        model_t = single_models[model_names[i]+'_t'][years]
        model_f = single_models[model_names[i]+'_f'][years]
        model_tcr = coef_arma_cis(model_t, model_f)
        rf_year_range = np.where((forcings['year'] >= model_years[i][0]) & (forcings['year'] <= model_years[i][1]))[0]
        for rf_num in range(1000):
            for obs_temps in (['hadcrut4', 'gistemp', 'noaa', 'berkeley', 'cowtan_way']):
                obs_f = rf_anthro[rf_num][rf_year_range]
                obs_t = annual_temps[obs_temps][years]
                obs_tcr = coef_arma_cis(obs_t, obs_f)
                model_monte_carlo = np.random.normal(model_tcr['coef'], model_tcr['sd'], 100)
                obs_monte_carlo = np.random.normal(obs_tcr['coef'], obs_tcr['sd'], 100)
                for j in range(100):
                    skill.append(model_skill(obs_monte_carlo[j], model_monte_carlo[j]))
        df = pd.DataFrame({'skill' : skill})
        skill_mean.append(df['skill'].mean())
        skill_median.append(df['skill'].median())        
        skill_5th.append(df['skill'].quantile(0.05))
        skill_95th.append(df['skill'].quantile(0.95))        
        timeframe.append(str(model_years[i]))
        model.append(str(model_names[i]))
    
    df = pd.DataFrame({'skill_mean' : skill_mean,
                       'skill_median' : skill_median,
                       'skill_5th' : skill_5th,
                       'skill_95th' : skill_95th,
                       'model' : model,
                       'timeframe' : timeframe})
    df.to_csv(interim_path+'simmod_tcr_skill_scores.csv')    
    
simmod_time_skill_scores(single_models, model_names, model_years, annual_temps)
simmod_tcr_skill_scores(single_models, model_names, model_years, annual_temps, forcings)   

Analyzing SimMod2.6 hadcrut4 diffs from 1970 to 2017
Analyzing SimMod2.6 gistemp diffs from 1970 to 2017
Analyzing SimMod2.6 noaa diffs from 1970 to 2017
Analyzing SimMod2.6 berkeley diffs from 1970 to 2017
Analyzing SimMod2.6 cowtan_way diffs from 1970 to 2017
Analyzing SimMod4.5 hadcrut4 diffs from 1970 to 2017
Analyzing SimMod4.5 gistemp diffs from 1970 to 2017
Analyzing SimMod4.5 noaa diffs from 1970 to 2017
Analyzing SimMod4.5 berkeley diffs from 1970 to 2017
Analyzing SimMod4.5 cowtan_way diffs from 1970 to 2017
Analyzing SimMod6.0 hadcrut4 diffs from 1970 to 2017
Analyzing SimMod6.0 gistemp diffs from 1970 to 2017
Analyzing SimMod6.0 noaa diffs from 1970 to 2017
Analyzing SimMod6.0 berkeley diffs from 1970 to 2017
Analyzing SimMod6.0 cowtan_way diffs from 1970 to 2017
Analyzing SimMod8.5 hadcrut4 diffs from 1970 to 2017
Analyzing SimMod8.5 gistemp diffs from 1970 to 2017
Analyzing SimMod8.5 noaa diffs from 1970 to 2017
Analyzing SimMod8.5 berkeley diffs from 1970 to 2017
Analyzi

In [9]:
def simmod_obs_forcing_diffs(single_models, model_names, model_years, forcings, annual_temps):
    coef_mean, coef_sd, ci_mean, timeframe, model = [], [], [], [], []
    for i in range(len(model_names)):
        print('Analyzing '+model_names[i]+' diffs from '+str(model_years[i][0])+' to '+str(model_years[i][1]))
        coef, ci_lower, ci_upper, rf_number, obs_series = [], [], [], [], []
        model_year_range = single_models['year'].between(model_years[i][0], model_years[i][1])
        rf_anthro = np.swapaxes(forcings['rf_anthro'],0,1)   
        rf_year_range = np.where((forcings['year'] >= model_years[i][0]) & (forcings['year'] <= model_years[i][1]))[0]
        temp_year_range = np.where((annual_temps['year'] >= model_years[i][0]) & (annual_temps['year'] <= model_years[i][1]))[0]
        model_temp = single_models[model_names[i]+'_t'][model_year_range]
        model_rf = single_models[model_names[i]+'_f'][model_year_range]

        for rf_num in range(1000):
            for obs_temps in (['hadcrut4', 'gistemp', 'noaa', 'berkeley', 'cowtan_way']):
                obs_rf = rf_anthro[rf_num][rf_year_range]
                obs_temp = annual_temps[obs_temps][temp_year_range]
                temp_diff = model_temp - obs_temp
                rf_diff = model_rf - obs_rf
                results = coef_arma_cis(temp_diff, rf_diff)
                coef.append(results['coef'])
                ci_lower.append(results['ci_lower'])
                ci_upper.append(results['ci_upper'])
                rf_number.append(rf_num)
                obs_series.append(obs_temps)
        df = pd.DataFrame({'coef' : coef,
                           'ci_lower' : ci_lower,
                           'ci_upper' : ci_upper,
                           'rf_number' : rf_number,
                           'obs_series' : obs_series})
        df['ci_val'] = df['coef'] - df['ci_lower']
        coef_mean.append(df['coef'].mean())
        coef_sd.append(df['coef'].std())
        ci_mean.append(df['ci_val'].mean())
        timeframe.append(str(model_years[i]))
        model.append(str(model_names[i]))    
    df = pd.DataFrame({'coef_mean' : coef_mean,
                       'coef_sd' : coef_sd,
                       'ci_mean' : ci_mean,
                       'model' : model,
                       'timeframe' : timeframe})
    uncertainty = ((df['coef_sd']*2)**2 + df['ci_mean']**2)**(0.5)
    df['coef_low'] = df['coef_mean'] - uncertainty
    df['coef_high'] = df['coef_mean'] + uncertainty
    df.to_csv(interim_path+'simmod_obs_forcing_trend_diffs.csv')

simmod_obs_forcing_diffs(single_models, model_names, model_years, forcings, annual_temps)

Analyzing SimMod2.6 diffs from 1970 to 2017
Analyzing SimMod4.5 diffs from 1970 to 2017
Analyzing SimMod6.0 diffs from 1970 to 2017
Analyzing SimMod8.5 diffs from 1970 to 2017
